In [1]:
!pip install qiskit
!pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 106.8 MB/s eta 0:00:00


In [2]:
# Deutsch–Jozsa Algorithm using Qiskit 2.x
# Compatible with Qiskit >= 2.0.0

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt


In [3]:


!pip install qiskit qiskit_aer qiskit_ibm_runtime

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime import QiskitRuntimeService
import matplotlib.pyplot as plt


# ==========================================
# TASK 1 – Modify the Oracle
# ==========================================
def oracle_constant(qc, ancilla, value=0):
    """Constant oracle: f(x)=0 or f(x)=1"""
    if value == 1:
        qc.x(ancilla)

def oracle_balanced_custom(qc, inputs, ancilla):
    """Balanced oracle: flips ancilla for half of inputs"""
    for i in range(0, len(inputs), 2):
        qc.cx(inputs[i], ancilla)


def deutsch_jozsa_circuit(n, oracle_func, *oracle_args):
    """Deutsch–Jozsa Quantum Circuit"""
    qreg = QuantumRegister(n + 1, "q")
    creg = ClassicalRegister(n, "c")
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    # Step 1: Initialize |0...0>|1>
    qc.x(ancilla)
    qc.h(qreg)

    # Step 2: Oracle
    oracle_func(qc, *oracle_args)

    # Step 3: Apply Hadamard to input qubits
    for q in inputs:
        qc.h(q)

    # Step 4: Measure inputs
    qc.measure(inputs, creg)
    return qc


def run_dj(qc):
    """Run Deutsch–Jozsa circuit on simulator"""
    simulator = AerSimulator()
    tqc = transpile(qc, simulator)
    result = simulator.run(tqc, shots=1024).result()
    counts = result.get_counts()
    print("Measurement counts:", counts)
    plot_histogram(counts)
    plt.show()

    n = qc.num_clbits
    if counts.get("0" * n, 0) == 1024:
        print("Function is CONSTANT")
    else:
        print(" Function is BALANCED")


# --- Execute Task 1 ---
print("\n TASK 1: Modified Balanced Oracle")
n = 3
qc_balanced = deutsch_jozsa_circuit(n, oracle_balanced_custom, list(range(n)), n)
print(qc_balanced.draw(fold=-1))
run_dj(qc_balanced)


# ==========================================
# TASK 2 – Change the Number of Input Qubits
# ==========================================
print("\n⚙ TASK 2: Changing Number of Input Qubits")

for n in [2, 4, 5]:
    print(f"\nRunning Deutsch–Jozsa with {n} input qubits")
    qc_balanced = deutsch_jozsa_circuit(n, oracle_balanced_custom, list(range(n)), n)
    print(qc_balanced.draw(fold=-1))
    run_dj(qc_balanced)


# ==========================================
# TASK 3 – Add Noise Simulation
# ==========================================
print("\n TASK 3: Noise Simulation")

def run_dj_noisy(qc):
    noise_model = NoiseModel()
    simulator = AerSimulator(noise_model=noise_model)
    tqc = transpile(qc, simulator)
    result = simulator.run(tqc, shots=1024).result()
    counts = result.get_counts()
    print("Measurement counts (with noise):", counts)
    plot_histogram(counts)
    plt.show()

n = 3
qc_balanced_noisy = deutsch_jozsa_circuit(n, oracle_balanced_custom, list(range(n)), n)
run_dj_noisy(qc_balanced_noisy)


# ==========================================
# TASK 4 – Run on IBM Quantum Device
# ==========================================
print("\n TASK 4: Running on IBM Quantum Real Device")

# To run on IBM Quantum, you must have an IBM account and API token.
# (You can skip this block if running offline.)

try:
    service = QiskitRuntimeService(channel="ibm_quantum")
    backend = service.backend("ibmq_qasm_simulator")

    qc_real = deutsch_jozsa_circuit(3, oracle_balanced_custom, list(range(3)), 3)
    tqc = transpile(qc_real, backend)
    job = backend.run(tqc, shots=1024)
    result = job.result()
    counts = result.get_counts()
    print("IBM Quantum backend counts:", counts)
    plot_histogram(counts)
    plt.show()
except Exception as e:
    print("⚠ IBM Quantum execution skipped or not available:", e)


# ==========================================
# TASK 5 – Circuit Analysis
# ==========================================
print("\n TASK 5: Circuit Analysis")

def analyze_oracle():
    qc = QuantumCircuit(3)
    oracle_balanced_custom(qc, [0, 1, 2], 1)
    print("\nOracle definition matrix:")
    print(qc.to_gate().definition)

analyze_oracle()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.4/377.4 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.9 MB/s eta 0:00:00

🧩 TASK 1: Modified Balanced Oracle
     ┌───┐          ┌───┐          ┌─┐   
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───
     ├───┤┌───┐  │  └┬─┬┘          └╥┘   
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║ ┌─┐
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐└───┘ ║ └╥┘
q_3: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├──────╫──╫─
     └───┘└───┘└───┘  ║  └───┘      ║  ║ 
c: 3/═════════════════╩═════════════╩══╩═
                      1             0  2 
Measurement counts: {'101': 1024}
✅ Function is BALANCED

⚙ TASK 2: Changing Number of Input Qubits

Running Deutsch–Jozsa with 2 input qubits
     ┌───┐          ┌───┐┌─┐
q_0: ┤ H ├────